In [1]:
import keras
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import warnings

from collections import Counter

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.metrics import top_k_categorical_accuracy
from keras.callbacks import EarlyStopping
from keras import regularizers
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, MultiLabelBinarizer
from sklearn.utils import class_weight
from os import listdir

from imblearn.over_sampling import RandomOverSampler, SMOTE

C:\Users\VuDinhPhong\Anaconda3\envs\YOLO-env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)


In [3]:
TRAINING_IMG_FOLDER = os.path.join(os.getcwd(), 'train')
TEST_IMG_FOLDER = os.path.join(os.getcwd(), 'test')


RESIZE_WIDTH = 178
RESIZE_HEIGH = 178

INPUT_WIDTH = 178
INPUT_HEIGHT = 178

BATCH_SIZE = 64
EPOCH = 200

FIG_SIZE = 15

MODE = 'TRAIN'

In [4]:
train_csv = pd.read_csv('train.csv')
train_csv.head()

,Image,Id
0,00022e1a.jpg,w_e15442c
1,000466c4.jpg,w_1287fbc
2,00087b01.jpg,w_da2efe0
3,001296d5.jpg,w_19e5482
4,0014cfdf.jpg,w_f22f3e3


In [5]:
def load_imgs_from_folder(folder, filenames=None):
    """
        Read images into a numpy array
        Input: 
            - folder: a path
            - filenames: list of images name in folder 
    """
    imgs = []
    file_names = filenames if filenames is not None else train_csv.Image
    for f in file_names:
        img = cv2.imread(os.path.join(folder, f))
        img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGH))
        img = img.reshape(1, RESIZE_WIDTH, RESIZE_HEIGH, 3)
        imgs.append(img)
    imgs = np.vstack(imgs)
    return imgs

def load_data_from_file(hdf5_file):
    """
        Load numpy array from hdf5
        
    """
    raise NotImplementedError
    

In [6]:
def plot_imgs(imgs_list, imgs_name=None):
    
    fig = plt.figure(figsize=(FIG_SIZE,FIG_SIZE))
    if imgs_name is None:
        imgs_name = np.arange(len(imgs_list))
        
    _COL = 5
    _ROW = (len(imgs_list)/_COL) + 1
    
    if not isinstance(imgs_list, list):
        imgs_list = [imgs_list]
    
    for i,im in enumerate(imgs_list):
        
        ax = fig.add_subplot(_ROW, _COL, i+1)
        
        ax.set_title(imgs_name[i])
        ax.imshow(im, cmap='gray')

In [7]:
class LabelOntHotEncoder():
    """
         A class which transform labels data to ont hot encoding
    """
    
    def __init__(self, Y):
        """
            Initialize with training label Y
        """
        if not any(Y):
            raise ValueError('Intialize error, missing Y')

        self.label_encoder = LabelEncoder()
        self.label_binarizer = LabelBinarizer()
        self.y_encoded = self.label_encoder.fit_transform(Y)
        self.y_one_hot = self.label_binarizer.fit_transform(self.y_encoded)
    
    def Y_one_hot(self):
        return self.y_one_hot
    
    def encode(self, y):
        """
            Input y: array of shape (N, ) contains category of class like 'new_whale', ...
            return: One hot like (N, num_class)
        """
        y_encoded = self.label_encoder.transform(y)
        return self.label_binarizer.transform(y_encoded)
    
    def decode(self, y):
        """
            Input y: numpy array of shape (N, num_class) contains ont not like [ [0,1,0,0,...], [1,0,0,0,...] ]
            return: numpy array of string (N, )
        """
        y_decoded = self.label_binarizer.inverse_transform(y)
        return self.label_encoder.inverse_transform(y_decoded)
    
    def inverse_label(self, y):
        """
            Input y: numpy array of shape (N,) contains position of top k class
            return: numpy array of string (N, )
        """
        return self.label_encoder.inverse_transform(y)
    
    def num_class(self):
        
        return len(self.label_binarizer.classes_)
    
    def classes(self):
        
        return self.label_binarizer.classes_

In [8]:
x = load_imgs_from_folder(TRAINING_IMG_FOLDER, train_csv.Image)
x.shape

(9850, 178, 178, 3)

In [12]:
res = RandomOverSampler(ratio='not minority')
x, y = res.fit_sample(x.reshape(-1, RESIZE_WIDTH*RESIZE_HEIGH*3), train_csv.Id.values)

x = x.reshape(len(x), RESIZE_WIDTH, RESIZE_HEIGH, 3)
print(Counter(y).items())

MemoryError: 

In [10]:
label_encoder = LabelOntHotEncoder(y)
y_one_hot = label_encoder.Y_one_hot()
print(y_one_hot.shape)

class_rebalance = class_weight.compute_class_weight('balanced', np.unique(train_csv.Id.values), train_csv.Id.values) 
class_rebalance.shape

(10659, 4251)


(4251,)

In [11]:
label_encoder.inverse_label([12, 13, 54, 0])

array(['w_00d8453', 'w_00fbb4e', 'w_02d7dc8', 'new_whale'], dtype=object)

In [12]:
train_datagen = ImageDataGenerator(featurewise_center=True,  
                                   featurewise_std_normalization=True, 
                                   rotation_range=20, rescale=1./255, 
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split = 0.2,)
train_datagen.fit(x)


In [13]:
def insert_intermediate_layer_in_keras(model, layer_id, new_layer):
    from keras.models import Model

    layers = [l for l in model.layers]

    x = layers[0].output
    for i in range(1, len(layers)):
        if i == layer_id:
            x = new_layer(axis=-1)(x)
        x = layers[i](x)

    new_model = Model(inputs=layers[0].input, outputs=x)
    return new_model

In [ ]:
def base_model():
    model = Sequential()

    model.add(Conv2D(input_shape=(RESIZE_WIDTH, RESIZE_HEIGHT, 3), filters=32, kernel_size=3, ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D())

    model.add(Conv2D(filters=64, kernel_size=3, ))
    model.add(BatchNormalization())

    model.add(Activation('relu'))
    model.add(MaxPool2D())

    model.add(Conv2D(filters=64, kernel_size=3, ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D())

    model.add(Conv2D(filters=64, kernel_size=3, ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D())

    model.add(Conv2D(filters=64, kernel_size=3, ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D())

    model.add(Flatten())
    model.add(Dense(1000))
    model.add(Activation('relu'))
    model.add(Dense(label_encoder.num_class(),activation='softmax'))
    
    model.compile(loss=categorical_crossentropy, 
                  metrics=[top_5_categorical_accuracy, 'accuracy'], optimizer='adam')
    
    return model

In [14]:
vgg_net = VGG16(include_top=False, weights=None, input_tensor=None, input_shape=(INPUT_WIDTH,INPUT_HEIGHT,3))

# Adding fully-connected at the end
flatten = Flatten()(vgg_net.output)
fully = Dense(4096, activation='relu', kernel_initializer='random_uniform',
              kernel_regularizer=regularizers.l2(0.0001))(flatten)
preds = Dense(label_encoder.num_class(), activation='softmax', kernel_initializer='random_uniform')(fully)
vgg_net = Model(vgg_net.input, preds)

# Adding BatchNorm
vgg_net = insert_intermediate_layer_in_keras(vgg_net, 10, BatchNormalization)
vgg_net = insert_intermediate_layer_in_keras(vgg_net, 14, BatchNormalization)
vgg_net = insert_intermediate_layer_in_keras(vgg_net, 18, BatchNormalization)



vgg_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 178, 178, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 178, 178, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 178, 178, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 89, 89, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 89, 89, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 89, 89, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 44, 44, 128)       0         
__________

In [15]:
def top_5_categorical_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5) 

In [16]:
early_stopping_20 = EarlyStopping(patience=20)

In [17]:
vgg_net.compile(optimizer=Adam(lr=0.0001), 
                loss=categorical_crossentropy, 
                metrics=[top_5_categorical_accuracy, 'accuracy'], )

In [18]:
if MODE == 'TRAIN': 
    vgg_net.fit_generator(generator=train_datagen.flow(x, y_one_hot, batch_size=BATCH_SIZE, subset='training'),
                          steps_per_epoch=x.shape[0]/BATCH_SIZE, epochs=EPOCH, 
                          validation_data=train_datagen.flow(x, y_one_hot, batch_size=BATCH_SIZE, subset='validation'), 
                          callbacks=[early_stopping_20], 
                          class_weight=class_rebalance)
    vgg_net.save_weights("vgg_16_batch_norm")
elif MODE == 'TEST':
    vgg_net.load_weights('vgg_16_batch_norm')

Epoch 1/200
167/166 [==============================] - 117s 701ms/step - loss: 11.5701 - top_5_categorical_accuracy: 0.1597 - acc: 0.1498 - val_loss: 17.7104 - val_top_5_categorical_accuracy: 0.0089 - val_acc: 9.3853e-04
Epoch 2/200
167/166 [==============================] - 108s 648ms/step - loss: 10.5163 - top_5_categorical_accuracy: 0.1814 - acc: 0.1668 - val_loss: 17.6636 - val_top_5_categorical_accuracy: 0.0211 - val_acc: 4.6926e-04
Epoch 3/200
 60/166 [=========>....................] - ETA: 1:02 - loss: 10.0685 - top_5_categorical_accuracy: 0.1919 - acc: 0.1677

KeyboardInterrupt: 

In [ ]:
def make_submission(test_folder, test_filenames, model, save_name='sbmission.csv'):
    test_array = load_imgs_from_folder(test_folder, filenames=test_filenames)
    test_predict = model.predict(test_array)
    test_argsort = (-test_predict).argsort(axis=1)[:,:5]
    df_submit = pd.DataFrame(columns=['Image', 'Id'])
    
    test_filenames = listdir(TEST_IMG_FOLDER)
    for i, predicted_ in enumerate(test_argsort):
        labels = " ".join(label_encoder.inverse_label(predicted_))
        df_submit = df_submit.append({'Image': test_filenames[i],
                         'Id': labels}, ignore_index=True)
    df_submit.to_csv('phongvu95_submission.csv', index=False)

In [ ]:
make_submission(TEST_IMG_FOLDER, listdir(TEST_IMG_FOLDER), vgg_net)